In [ ]:
# pip install tensorflow opencv-python mediapipe sklearn matplotlib

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
from matplotlib import pyplot as plt

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [21]:
import json

with open('archive/WLASL.json') as f:
    data = json.load(f)

counter2 = 0
for i in data:
    gloss = i['gloss']
    counter = 0
    for inst in i['instances']:
        split = inst['split']

        if split == 'train':
            videoID = inst['video_id']
            # check if video exists
            if os.path.isfile('archive/videos/' + videoID + '.mp4'):
                print("Video ID : ", videoID)
                with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                    cap = cv2.VideoCapture('archive/videos/' + videoID + '.mp4')
                    frameRate = cap.get(2)
                    filename = 0
                    while (cap.isOpened()):
                        frameId = cap.get(1)
                        ret, frame = cap.read()
                        if ret != True:
                            break
                        if (frameId % ((int(frameRate)+1)) == 0):
                            image, results = mediapipe_detection(frame, holistic)
                            draw_styled_landmarks(image, results)
                            length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                            keypoints = extract_keypoints(results)
                            npy_path = os.path.join("train", gloss, str(counter) ,str(frameId) + ".npy")
                            if not os.path.exists(os.path.dirname(npy_path)):
                                os.makedirs(os.path.dirname(npy_path))
                            np.save(npy_path, keypoints)
                    counter+=1
            else:
                print("Video ID : ", videoID, " does not exist")
                    
    cap.release()
    cv2.destroyAllWindows()
    
    if counter2 == 2:
        break
    counter2+=1


Video ID :  69241
Video ID :  65225  does not exist
Video ID :  68011  does not exist
Video ID :  68208  does not exist
Video ID :  68012  does not exist
Video ID :  70266  does not exist
Video ID :  07085  does not exist
Video ID :  07086  does not exist
Video ID :  07087  does not exist
Video ID :  07069
Video ID :  07088  does not exist
Video ID :  07089  does not exist
Video ID :  07090  does not exist
Video ID :  07091  does not exist
Video ID :  07068
Video ID :  07097  does not exist
Video ID :  07070
Video ID :  07071  does not exist
Video ID :  07073  does not exist
Video ID :  67424  does not exist
Video ID :  07074
Video ID :  07075  does not exist
Video ID :  07076  does not exist
Video ID :  07077  does not exist
Video ID :  07078  does not exist
Video ID :  07079  does not exist
Video ID :  07080  does not exist
Video ID :  07081  does not exist
Video ID :  07082  does not exist
Video ID :  07084  does not exist
Video ID :  65539
Video ID :  68660  does not exist
Video ID

In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
actions = ["book", "computer", "drink"]
label_map = {label:num for num, label in enumerate(actions)}